## 郵局特種郵件追蹤查詢資料

### 資料內容
V1          | V2      | V3     | V4      | V5       |V6
------------|:-------:|-------:| -------:|---------:|-------
郵件狀態代碼  | 掛號號碼 | 處理日期 | 處理時間 | 處理局號  | 其他 

> 此檔案一共含有三億八千萬筆資料，因此為了確寶電腦可以運行，將資料先切割成90份，再依序讀入。  
> 郵件狀態代碼是在解釋郵件在不同時期的狀況，例如：交寄郵件、投遞成功、投遞不成功等等。  
> 在這邊先整理資料，計算每個郵局（處理局號）經手的郵件的狀態數量。  
> 計算完後再將新的dataframe輸出成csv檔，以利之後直接讀入資料。

In [ ]:
library(forcats)

In [ ]:
#工作目錄
setwd("C:\\Users\\chrisyu\\Desktop\\postdata\\tt\\")
code <- c("A1",	"A2",	"A3",	"G1",	"G2",	"G3",	"G4",	"G5",	"G7",	"G8",	"G9",	"H4",	"H7",	"HL",	"I1",	"I2",	"I3",	"I4",	"I5",	"I6",	"I7",	"I8",	"I9",	"IL",	"L1",	"M1",	"N1",	"O1",	"P3",	"P4",	"P5",	"Q1",	"S1",	"S2",	"T1",	"T2",	"T3",	"T4",	"T5",	"T6",	"U2",	"V1",	"V2",	"V3",	"V4",	"V5",	"V6",	"VL",	"W2",	"W7",	"X2",	"X7",	"Y1",	"Y4",	"Y5",	"Y7",	"Z1",	"Z2",	"Z4",	"Z5",	"Z7",	"Z8",	"Z9")

postall <- NULL
#讀入資料
for(i in c(0:89)){
  post <- NULL
  i <- formatC(i, width = 2, flag = 0) #函数是把1轉化成01
  path <- paste0("tt",i)
  ttcsv <- read.csv(file = path, header = FALSE)
  ttcsv$V5 <- as.factor(ttcsv$V5)
  post <- data.frame(levels(ttcsv$V5))
  colnames(post) <- c('office')
#判斷狀態並計算數量
  for(b in code){
    ttsub <- subset(ttcsv, V1 == b)
    ft <- fct_count(ttsub$V5)
    ft <- as.data.frame(ft)
    colnames(ft) <- c('office',b)
    post <- merge(x=post,y=ft,by='office')
  }
#將不同檔案內dataframe合併成最終的postall
  diffoff <- setdiff(post$office,postall$office)
  plevel <- levels(as.factor(c(diffoff,levels(postall$office))))
  postall$office <- factor(postall$office, levels=plevel)
  post$office <- factor(post$office, levels=plevel)
  for(j in c(1:dim(post)[1])){
    ind <- which(postall$office==post$office[j])
    if (length(ind)>0){
      sumtable <- rbind(postall[ind,-1],post[j,-1])
      sumtable <- rbind(sumtable,colSums(sumtable))
      officetable <- data.frame(c(as.character(postall[ind,1]),as.character(postall[ind,1]),as.character(postall[ind,1])))
      colnames(officetable) <- c('office')
      sumtable <- cbind(officetable,sumtable)
      postall[ind,] <- sumtable[3,]
    }else{
      postall <- rbind(postall,post[j,])
    }
  }
}
#輸出postall檔案
write.table(postall,file="C:\\Users\\chrisyu\\Desktop\\postdata\\PostOffice.csv",sep=",",row.names=F, na = "NA")
